### Transfer learning
A machine learning method where a model developed for a first task is then reused as the starting point for a model on a second task.

例如，我们可以训练一个模型来识别鸟类和猫，然后可以用同一个模型，只对最后一个 layer 进行一些修改，就能使用这个新的模型去对蜜蜂和狗进行分类。

所以这是深度学习中一个很受欢迎的生成新的模型的方法，能节省大量的时间。

### Image Folder
对于一个数据库，内部数据是按文件包的形式分别存储的，就要使用 datasets.ImageFolder 来加载数据。

比如一个数据库文件包里包含两个文件包：train 和 test，每个文件下又分别包含不同的类的数据，比如 ants 和 bees。

In [1]:
# ImageFolder
# Scheduler
# Transfer Learning

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# hyper parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [4]:
# transforms
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [5]:
# import data
data_dir = 'data/hymenoptera_data'
sets = ['train', 'val']
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['trian', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

FileNotFoundError: [Errno 2] No such file or directory: 'data/hymenoptera_data/train'

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs-1}")
        print('-' * 10)
        
        # each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                shceduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")
    print()
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features # num of input features of the last layer

model.fc = nn.Linear(num_features, 2)
model.to(device)

criterion == nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# scheduler
# this will update the learning rate
# every {step_size} epochs, learning rate will be updated by {gamma} (percentagely)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epoch=2)

In [ ]:
model = models.resnet18(pretrained=True)
for parameter in model.parameters():
    parameter.requires_grad = False

num_features = model.fc.in_features # num of input features of the last layer

model.fc = nn.Linear(num_features, 2)
model.to(device)

criterion == nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# scheduler
# this will update the learning rate
# every {step_size} epochs, learning rate will be updated by {gamma} (percentagely)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epoch=2)